# SECTION 02 重要箇所をテスト実装する
- このセクションで学ぶこと
  - なぜテスト実装が必要なのか
  - URLを指定してHTMLを取得
  - 正規表現でHTMLからURLを取得
  - 相対URLを絶対URLに変換
  - URLが画像ファイルか否かを判定
  - 画像ファイルを保存

## なぜテスト実装が必要なのか
- 開発スピードを向上させるため
  - ⼤きなサービスのなかで⼩さな1つの機能を0から開発するのは「起動」「準備(テスト値の⽤意など)」に時間がかかる
  - ⼩さなコードでエラーを繰り返しながら機能を実現させ、そのあとで組み込めばトラブルが少ない
- 当初の想定と異なる場合があるため
  - 利⽤を想定していたライブラリが⽬的の機能を果たせない場合などがある
  - 他を作り込んだうえで発覚すると修正が⼤変なので⼤枠を固める前に可能な限りパーツを作っておく
- すでに重要箇所が動いているとメンタル的に安⼼して開発できる

## URLを指定してHTMLを取得
- 本番プログラムは⼤きくて複雑なので、「本番でも使いやすいテスト⽤のコード(たとえば関数)」を作成するのがよい
- HTMLの取得には6章のrequestsモジュールを利⽤

In [1]:
import requests
def download_html(url):
    response = requests.get(url)
    html = response.text
    return html

html = download_html('https://gihyo.jp/book/list')
fout = open('sample.html', 'w', encoding='utf8')
fout.write(html)
fout.close()

## 正規表現でHTMLからURLを取得
- 先に取得したHTML内にはURLが多数ある
- ウェブサイトや画像のURLを抜き出す処理に正規表現を使う
- 「正規表現」は 特定のパターンの⽂字列が含まれるかのチェックや抜き出し処理をする⼿法。他のプログラミング⾔語にもある
- 複雑な正規表現のパターンは検索するのが⼿っ取り早い。たとえば「Python 正規表現 URL」や「Python regex URL」など

In [3]:
import re
text = '<p>Hello World</p><a href="http://example.com">More Examples</a><a href="http://example2.com">Even More Examples</a>"'
links = re.findall('<a href="?\'?([^"\'>]*)', text)
print(links)

['http://example.com', 'http://example2.com']


In [6]:
import requests
import re
def get_urls(url):
    response = requests.get(url)
    html = response.text
    urls = re.findall('<a href="?\'?([^"\'>]*)', html)
    return urls
urls = get_urls('https://gihyo.jp/book/list')
#print(urls)

## 相対URLを絶対URLに変換
- 絶対URL: https://gihyo.jp/news/info/2020/03/1301
- 相対URL(ドメインが省略): /news/info/2020/03/1301
- 相対URLは現在のURLを基準とした表記なので、プログラムで使う場合は現在地に依存がない絶対URLへの変換が望ましい

In [7]:
import urllib.parse
def get_abs_url(current_url, url):
    abs_url = urllib.parse.urljoin(current_url, url)
    return abs_url
print(get_abs_url('https://gihyo.jp/book/list', 'http://gihyo.jp/site/inquiry'))
print(get_abs_url('https://gihyo.jp/book/list', '/book/2017/978-4-7741-8751-8'))
print(get_abs_url('https://gihyo.jp/book/list', '2017/978-4-7741-8751-8'))

http://gihyo.jp/site/inquiry
https://gihyo.jp/book/2017/978-4-7741-8751-8
https://gihyo.jp/book/2017/978-4-7741-8751-8


## URLが画像ファイルか否かを判定
- URLを以下の2種類のいずれか拡張⼦で判定する
  - 画像ファイル: 拡張⼦が「.jpg」「.png」「.gif」
  - HTMLファイル: 画像ファイルでない場合全て
- (厳密にはHTTPレスポンスのMIMEで判断することが望ましい)

In [8]:
def check_url_type(url):
    if '.jpg' in url:
        print('jpg')
    elif '.png' in url:
        print('png')
    elif '.gif' in url:
        print('gif')
    else:
        print('html')

## 画像ファイルを保存
- requestsで取得したレスポンスからデータを取得
- データをバイナリ書き込みモードでファイルに保存

In [9]:
import requests
import os
def save_image(url, filepath):
    # 画像データを取得
    response = requests.get(url, stream=True)
    image_data = response.content
    fout = open(fpath, 'wb')
    fout.write(image_data)
    fout.close()

## 演習
- このセクションで作成したプログラムを全て実⾏してみる
- 関数定義のみで実⾏コードが書かれていないプログラムは、⾃分で実⾏コードを作成してください